In [ ]:
pip install selenium


In [ ]:
pip install webdriver-manager


In [27]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains  # Import ActionChains

# Initialize WebDriver
driver = webdriver.Chrome()
url = "https://www.imdb.com/search/title/?title_type=feature&release_date=2024-01-01,2024-12-31&genres=game-show"
driver.get(url)

# Wait for the first page to load
time.sleep(3)

# Lists to store movie details
titles, ratings, votings, durations = [], [], [], []

# Define click_load_more function BEFORE using it
def click_load_more(): 
    try: 
        load_more_button = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/div[2]/div/span/button/span/span')
        ActionChains(driver).move_to_element(load_more_button).perform()  # Scroll to the button
        load_more_button.click()
        time.sleep(3)  # Allow content to load
        return True 
    except Exception as e: 
        print("Error clicking 'Load More' button:", e) 
        return False 

# Main scraping loop
while True:
    try:
        # Wait for movies to load
        WebDriverWait(driver, 10).until(              #//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/div[2]/div/span/button/span
            EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/ul/li'))
        )

        # Click "Load More" until no more content appears
        while click_load_more(): 
            print("Clicked 'Load More' button")
        print("No more content to load.")

        # Scroll to bottom to load all content
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Allow content to load

        # Get movie elements
        movie_items = driver.find_elements(By.XPATH, '//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/ul/li')

        for movie_item in movie_items:
            try:
                # Extract movie title
                title = movie_item.find_element(By.XPATH, './/div/div/div/div[1]/div[2]/div[1]/a/h3').text

                # Extract rating (handle missing values)
                rating = movie_item.find_element(By.XPATH, './/div/div/div/div[1]/div[2]/span/div/span/span[1]').text if movie_item.find_elements(By.XPATH, './/div/div/div/div[1]/div[2]/span/div/span/span[1]') else "N/A"

                # Extract voting (handle missing values)
                voting = movie_item.find_element(By.XPATH, './/div/div/div/div[1]/div[2]/span/div/span/span[2]').text if movie_item.find_elements(By.XPATH, './/div/div/div/div[1]/div[2]/span/div/span/span[2]') else "N/A"

                # Extract duration (handle missing values)
                duration = movie_item.find_element(By.XPATH, './/div/div/div/div[1]/div[2]/div[2]/span[2]').text if movie_item.find_elements(By.XPATH, './/div/div/div/div[1]/div[2]/div[2]/span[2]') else "N/A"

                # Append to lists (avoid duplicates)
                if title not in titles:
                    titles.append(title)
                    ratings.append(rating)
                    votings.append(voting)
                    durations.append(duration)

            except Exception as e:
                print(f"Error extracting a movie: {e}")
                continue

        # Try clicking the "Next Page" button
        try:
            next_button_xpath = '//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/div[2]/div/span/button/span/span'
            next_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, next_button_xpath))
            )

            # Scroll to "Next" button and click it
            driver.execute_script("arguments[0].scrollIntoView();", next_button)
            time.sleep(1)  # Small delay before clicking
            next_button.click()
            time.sleep(5)  # Allow new page content to load

        except (TimeoutException, NoSuchElementException):
            print("No more pages. Scraping complete.")
            break  # Exit loop if Next button is unavailable

    except Exception as e:
        print(f"Error navigating pages: {e}")
        break  # Break on major errors

# Create DataFrame
df = pd.DataFrame({
    'Title': titles, 
    'Rating': ratings,
    'Votes': votings,
    'Duration': durations
})

# Save to CSV
file_path = "imdb_2024_movies_game-show.csv"
df.to_csv(file_path, index=False, mode="w", encoding="utf-8")

print(f"Scraped {len(df)} movies successfully! File saved as: {file_path}")

# Close browser
driver.quit()



Error clicking 'Load More' button: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/div[2]/div/span/button/span/span"}
  (Session info: chrome=134.0.6998.166); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6121D4C25+3179557]
	(No symbol) [0x00007FF611E388A0]
	(No symbol) [0x00007FF611CC91CA]
	(No symbol) [0x00007FF611D1FA67]
	(No symbol) [0x00007FF611D1FC9C]
	(No symbol) [0x00007FF611D73627]
	(No symbol) [0x00007FF611D47C6F]
	(No symbol) [0x00007FF611D702F3]
	(No symbol) [0x00007FF611D47A03]
	(No symbol) [0x00007FF611D106D0]
	(No symbol) [0x00007FF611D11983]
	GetHandleVerifier [0x00007FF6122367CD+3579853]
	GetHandleVerifier [0x00007FF61224D1D2+3672530]
	GetHandleVerifier [0x00007FF612242153+3627347]
	GetHandleVe

In [1]:
import pandas as pd
import os

# Path where genre-wise CSVs are stored
csv_folder = r"C:\Users\SIBI\OneDrive\Desktop\web_Scraping_selenium (2)\web_Scraping_selenium\guvi"

# Get all CSV file names
csv_files = [f for f in os.listdir(csv_folder) if f.endswith('.csv')]

# Read and merge all CSVs with genre information
df_list = []

for file in csv_files:
    genre_name = os.path.splitext(file)[0]  # Extract genre from filename
    df = pd.read_csv(os.path.join(csv_folder, file))
    df['genre'] = genre_name  # Add genre column
    df_list.append(df)

# Concatenate all DataFrames
merged_df = pd.concat(df_list, ignore_index=True)

# Remove duplicates (if necessary)
merged_df.drop_duplicates(inplace=True)

# Save merged data
output_file = os.path.join(csv_folder, "all_movies_2024.csv")
merged_df.to_csv(output_file, index=False)

# Display the first few rows
print(merged_df.head())

print(f"Merged CSV saved at: {output_file}")



                          Title  Rating     Votes Duration   genre
0          1. Kraven the Hunter     5.4   46000.0    2h 7m  action
1                   2. Twisters     6.5  161000.0    2h 2m  action
2               3. Gladiator II     6.5  218000.0   2h 28m  action
3       4. Sonic the Hedgehog 3     6.9   53000.0   1h 50m  action
4  5. Pushpa: The Rule - Part 2     6.1   54000.0   3h 21m  action
Merged CSV saved at: C:\Users\SIBI\OneDrive\Desktop\web_Scraping_selenium (2)\web_Scraping_selenium\guvi\all_movies_2024.csv


In [52]:
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect("movies_2024.db")

# Query to get unique movies with their proper information
query = """
SELECT DISTINCT Title, genre, Rating, Votes, Duration 
FROM movies
WHERE genre != 'all_movies_2024'  -- Filter out the incorrect genre
ORDER BY Rating DESC, Votes DESC
LIMIT 10
"""

top_movies = pd.read_sql(query, conn)
print(top_movies)

conn.close()

                                    Title        genre  Rating Votes Duration
0        1500. Opioids: The Hidden Crisis  documentary    10.0   9.0     None
1  2516. Retratos de República Dominicana  documentary    10.0   9.0    1h 1m
2                      1010. The Premiere       comedy    10.0   8.0   1h 46m
3                        366. Clownface 3       action    10.0  34.0   1h 44m
4                        389. Clownface 3       horror    10.0  34.0   1h 44m
5                        156. Clownface 3      mystery    10.0  34.0   1h 44m
6                        616. Clownface 3     thriller    10.0  34.0   1h 44m
7                       437. Mashaarojinn       horror    10.0  18.0   1h 39m
8         65. Ice Cross: Life on the Edge        sport    10.0  11.0   1h 30m
9                 491. Azotes de Barrio 2       action     9.9   8.0     None


In [5]:
import streamlit as st
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Connect to SQLite database
def load_data():
    conn = sqlite3.connect("movies_2024.db")
    df = pd.read_sql("SELECT * FROM movies", conn)
    conn.close()
    return df

df = load_data()

st.title("IMDb 2024 Movies Analysis 🎬")
st.sidebar.header("Filter Options")

# Genre selection
selected_genre = st.sidebar.selectbox("Select Genre", ["All"] + sorted(df["genre"].unique()))
if selected_genre != "All":
    df = df[df["genre"] == selected_genre]


2025-03-31 12:49:21.431 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-31 12:49:21.434 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-31 12:49:21.437 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-31 12:49:21.437 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-31 12:49:21.443 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-31 12:49:21.444 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-31 12:49:21.447 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-31 12:49:21.452 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar